In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head(2)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0


In [6]:
df = df[ ['text','label_num'] ]
df.head(2)

,text,label_num
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0


In [7]:
!pip install nltk

In [13]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
# hediye -> HEDİYE ->
# Hediye, hediye
import string

ps = nltk.stem.porter.PorterStemmer()

def clean_text(text):
    text = text.lower() # Bütün harfleri küçük yap.

    text = text.translate(str.maketrans('','', string.punctuation)) # Bütün noktalama işaretlerini kaldır.

    # "Merhaba,Dünya."
    # ["merhaba","dünya"]
    words = text.split() # default => Boşluk Merhaba Dünya => ["Merhaba","Dünya"]   #Merhaba,dünya,abc -> ["Merhaba","dünya","abc"]

    words = [ word for word in words if not word.isdigit() ] # "500 TL" "4.5G" "3D" -> Yalnızca numerik kelimeler alınmayacak.
    # Stem -> Kelimeleri köküne indirgemek -> Running,Run -> RUN
    # Remove Stopwords -> The, is, at, on
    # Superhero - The Superhero
    words = [ ps.stem(word) for word in words if word not in nltk.corpus.stopwords.words('english') ]

    return " ".join(words)

In [22]:
df.head(3)

,text,label_num
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0


In [23]:
df['clean_text'] = df['text'].apply(clean_text)

In [24]:
df.head(3)

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\r\n...,0,subject enron methanol meter follow note gave ...
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,subject hpl nom januari see attach file hplnol...
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,subject neon retreat ho ho ho around wonder ti...


In [25]:
from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['label_num']

In [28]:
#Vectorize -> metni numerize etmek.
from sklearn.feature_extraction.text import TfidfVectorizer # TFIDF vectorizer nasıl çalışır?

In [32]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['clean_text'])
X_tfidf

<5171x37932 sparse matrix of type '<class 'numpy.float64'>'
	with 307420 stored elements in Compressed Sparse Row format>

In [33]:
# [ 
#  [ 0.8, 0.5, 0.3, 0 ,0 ,0],
#  [ 0.4, 1, 0.2, 0 , 0]
# ]

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

In [34]:
# Naive Bayes -> Bayes teoremi
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [40]:
y_pred = model.predict(X_test)

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
# precision -> kesinlik
# recall -> duyarlılık

# f1 score -> 

# support -> veri sayısı 0->1121 1->431

              precision    recall  f1-score   support

           0       0.89      1.00      0.94      1121
           1       1.00      0.68      0.81       431

    accuracy                           0.91      1552
   macro avg       0.95      0.84      0.88      1552
weighted avg       0.92      0.91      0.91      1552

